In [2]:
from haystack.telemetry import tutorial_running
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor
import logging
from haystack.nodes import PreProcessor
from langdetect import detect
import nltk
import pandas as pd



In [3]:
# Logs
logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [5]:
# Specify the file path to your CSV file
file_path = "dominio3.csv"

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
df.head()

,PageURL,Title,CreationDate,Seccion,Subseccion,Contenido,WikipageID,LastModified
0,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,NaN,NaN,Carbon dioxide is a chemical compound with the...,5906,2023-09-25T06:26:03Z
1,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,Chemical and physical properties,"Structure, bonding and molecular vibrations",The symmetry of a carbon dioxide molecule is l...,5906,2023-09-25T06:26:03Z
2,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,Chemical and physical properties,In aqueous solution,"Carbon dioxide is soluble in water, in which i...",5906,2023-09-25T06:26:03Z
3,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,Chemical and physical properties,Chemical reactions of CO2,CO2 is a potent electrophile having an electro...,5906,2023-09-25T06:26:03Z
4,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,Chemical and physical properties,Physical properties,Carbon dioxide is colorless. At low concentrat...,5906,2023-09-25T06:26:03Z


In [6]:
# Extraer las primeras 'numero_de_filas_a_extraer' filas y almacenarlas en un nuevo DataFrame
df_test = df.head(3)

df_test

,PageURL,Title,CreationDate,Seccion,Subseccion,Contenido,WikipageID,LastModified
0,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,NaN,NaN,Carbon dioxide is a chemical compound with the...,5906,2023-09-25T06:26:03Z
1,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,Chemical and physical properties,"Structure, bonding and molecular vibrations",The symmetry of a carbon dioxide molecule is l...,5906,2023-09-25T06:26:03Z
2,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,Chemical and physical properties,In aqueous solution,"Carbon dioxide is soluble in water, in which i...",5906,2023-09-25T06:26:03Z


In [ ]:
# Nombre de la columna que deseas convertir a archivos TXT
columna_a_convertir = 'Contenido'

# Recorrer cada elemento de la columna y crear archivos TXT
for indice, valor in enumerate(df_test[columna_a_convertir]):
    # Crear el nombre del archivo TXT usando el valor del elemento
    nombre_archivo = f"{valor}.txt"

    # Crear y escribir en el archivo TXT
    with open(nombre_archivo, 'w') as archivo:
        archivo.write(f"Nombre: {valor}\n")
        archivo.write(f"Edad: {df['Edades'][indice]}\n")

    print(f"Archivo {nombre_archivo} creado para {valor}")

# Verificación: Mostrar el contenido de los archivos TXT creados
for valor in df[columna_a_convertir]:
    nombre_archivo = f"{valor}.txt"
    with open(nombre_archivo, 'r') as archivo:
        contenido = archivo.read()
        print(f"Contenido de {nombre_archivo}:\n{contenido}\n")

In [3]:
# all_docs = convert_files_to_docs(dir_path=doc_dir)

converter = TextConverter(remove_numeric_tables=True, valid_languages=["en"])
doc_txt = converter.convert(file_path="prueba.txt", meta=None)[0]

In [9]:
from haystack.utils import convert_files_to_docs


all_docs = convert_files_to_docs(dir_path='tesis-qa-system')

In [4]:
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=True,
)

docs_default = preprocessor.process([doc_txt])
print(f"n_docs_input: 1\nn_docs_output: {len(docs_default)}")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sotof\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
Preprocessing: 100%|██████████| 1/1 [00:00<00:00, 66.56docs/s]

n_docs_input: 1
n_docs_output: 2


In [5]:
# Not respecting sentence boundary vs respecting sentence boundary

preprocessor_nrsb = PreProcessor(split_respect_sentence_boundary=False)
docs_nrsb = preprocessor_nrsb.process([doc_txt])

print("RESPECTING SENTENCE BOUNDARY")
end_text = docs_default[0].content[-50:]
print('End of document: "...' + end_text + '"')
print()
print("NOT RESPECTING SENTENCE BOUNDARY")
end_text_nrsb = docs_nrsb[0].content[-50:]
print('End of document: "...' + end_text_nrsb + '"')

Preprocessing: 100%|██████████| 1/1 [00:00<00:00, 663.13docs/s]

RESPECTING SENTENCE BOUNDARY
End of document: "..., is included as an eligible practice under EQIP. "

NOT RESPECTING SENTENCE BOUNDARY
End of document: "...EQIP funding is reserved for livestock operations."


In [6]:
# Sliding window approach

preprocessor_sliding_window = PreProcessor(split_overlap=3, split_length=10, split_respect_sentence_boundary=False)
docs_sliding_window = preprocessor_sliding_window.process([doc_txt])

doc1 = docs_sliding_window[0].content[:200]
doc2 = docs_sliding_window[1].content[:100]
doc3 = docs_sliding_window[2].content[:100]

print('Document 1: "' + doc1 + '..."')
print('Document 2: "' + doc2 + '..."')
print('Document 3: "' + doc3 + '..."')

Preprocessing: 100%|██████████| 1/1 [00:00<00:00, 500.39docs/s]

Document 1: "The EQIP covers up to 75 percent of costs incurred..."
Document 2: "of costs incurred and 100 percent of income foregone for..."
Document 3: "income foregone for implementing eligible conservation practices for awardees. There..."


In [11]:
all_docs = convert_files_to_docs(dir_path='tesis-qa-system')
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=True,
)
docs = preprocessor.process(all_docs)

print(f"n_files_input: {len(all_docs)}\nn_docs_output: {len(docs)}")

Preprocessing: 0docs [00:00, ?docs/s]

n_files_input: 0
n_docs_output: 0
